# Lab 07: Stacks and Queues

## Overview

This assignment involves the stack and queue abstract data types.

## Stack applications

The first two exercises ask you to use the stack data structure developed in class to develop two distinct stack-driven applications.

Below is the completed stack implementation from class. While you needn't modify it for this assignment — indeed, all tests run on our end will *not* make use of any changes you introduce to the `Stack` class — we urge you to read through the code and make sure you understand how it works.

In [1]:
class Stack:
    class Node:
        def __init__(self, val, next=None):
            self.val = val
            self.next  = next
    
    def __init__(self):
        self.top = None

    def push(self, val):
        self.top = Stack.Node(val, self.top)
        
    def pop(self):
        assert self.top, 'Stack is empty'
        val = self.top.val
        self.top = self.top.next
        return val
    
    def peek(self):
        return self.top.val if self.top else None
    
    def empty(self):
        return self.top == None
    
    def __bool__(self):
        return not self.empty()
    
    def __repr__(self):
        if not self.top:
            return ''
        return '--> ' + ', '.join(str(x) for x in self)
    
    def __iter__(self):
        n = self.top
        while n:
            yield n.val
            n = n.next

### 1. Paired delimiter matching

In class we wrote a function that uses a stack to help determine whether all paired delimiters (e.g., parentheses) in a given string are correctly matched — you can review the code at http://moss.cs.iit.edu/cs331/notebooks/stacks-and-queues.html (look for `check_parens`).

For this first exercise you will extend our implementation to check all the following paired delimiters: `{}, (), [], <>`. We've defined two strings — `delim_openers` and `delim_closers` — that might come in handy in your implementation (hint: look into using the `index` sequence method).  

In [2]:
delim_openers = '{([<'
delim_closers = '})]>'

def check_delimiters(expr):
    """Returns True if and only if `expr` contains only correctly matched delimiters, else returns False."""
    s = Stack()
    for c in expr:
        for i in delim_openers:
            if c == i:
                s.push(c)
                break
        else:
            for j in delim_closers:
                if c == j:
                    p = ''
                    if c == '}':
                        p = '{'
                    elif c == ')':
                        p = '('
                    elif c == ']':
                        p = '['
                    elif c == '>':
                        p = '<'
                    try:
                        assert s.pop() == p
                    except:
                        return False
    return s.empty()

In [3]:
# (1 point)

from unittest import TestCase
tc = TestCase()
tc.assertTrue(check_delimiters('()'))
tc.assertTrue(check_delimiters('[]'))
tc.assertTrue(check_delimiters('{}'))
tc.assertTrue(check_delimiters('<>'))

In [4]:
# (1 point)

from unittest import TestCase
tc = TestCase()
tc.assertTrue(check_delimiters('([])'))
tc.assertTrue(check_delimiters('[{}]'))
tc.assertTrue(check_delimiters('{<()>}'))
tc.assertTrue(check_delimiters('<({[]})>'))

In [5]:
# (2 points)

from unittest import TestCase
tc = TestCase()
tc.assertTrue(check_delimiters('([] () <> [])'))
tc.assertTrue(check_delimiters('[{()} [] (<> <>) {}]'))
tc.assertTrue(check_delimiters('{} <> () []'))
tc.assertTrue(check_delimiters('<> ([] <()>) <[] [] <> <>>'))

In [6]:
# (1 point)

from unittest import TestCase
tc = TestCase()
tc.assertFalse(check_delimiters('('))
tc.assertFalse(check_delimiters('['))
tc.assertFalse(check_delimiters('{'))
tc.assertFalse(check_delimiters('<'))
tc.assertFalse(check_delimiters(')'))
tc.assertFalse(check_delimiters(']'))
tc.assertFalse(check_delimiters('}'))
tc.assertFalse(check_delimiters('>'))

In [7]:
# (1 point)

from unittest import TestCase
tc = TestCase()
tc.assertFalse(check_delimiters('( ]'))
tc.assertFalse(check_delimiters('[ )'))
tc.assertFalse(check_delimiters('{ >'))
tc.assertFalse(check_delimiters('< )'))

In [8]:
# (2 points)

from unittest import TestCase
tc = TestCase()
tc.assertFalse(check_delimiters('[ ( ] )'))
tc.assertFalse(check_delimiters('((((((( ))))))'))
tc.assertFalse(check_delimiters('< < > > >'))
tc.assertFalse(check_delimiters('( [] < {} )'))

### 2. Infix &rarr; Postfix conversion

Another function we looked at was one that used a stack to evaluate a postfix arithmetic expression — you can review the code at http://moss.cs.iit.edu/cs331/notebooks/stacks-and-queues.html (look for `eval_postfix`). Because most of us are more accustomed to infix-form arithmetic expressions (e.g., `2 * (3 + 4)`), however, the function seems to be of limited use. The good news: we can use a stack to convert an infix expression to postfix form!

To do so, we will use the following algorithm:

1. Start with an empty list and an empty stack. At the end of the algorithm, the list will contain the correctly ordered tokens of the postfix expression.

2. Next, for each token in the expression (split on whitespace):

    - if the token is a digit (the string `isdigit` method can be used to determine this), simply append it to the list; else, the token must be either an operator or an opening or closing parenthesis, in which case apply one of the following options:

    - if the stack is empty or contains a left parenthesis on top, push the token onto the stack.

    - if the token is a left parenthesis, push it on the stack.

    - if the token is a right parenthesis, pop the stack and append all operators to the list until a left parenthesis is popped. Discard the pair of parentheses.

    - if the token has higher precedence than the top of the stack, push it on the stack. For our purposes, the only operators are +, -, *, /, where the latter two have higher precedecence than the first two.

    - if the token has equal precedence with the top of the stack, pop and append the top of the stack to the list and then push the incoming operator.

    - if the incoming symbol has lower precedence than the symbol on the top of the stack, pop the stack and append it to the list. Then repeat the above tests against the new top of stack.

3. After arriving at the end of the expression, pop and append all operators on the stack to the list.

A writeup containing a detailed explanation of the steps above (though it prints the tokens immediately rather than adding them to a list) can be found at http://csis.pace.edu/~wolf/CS122/infix-postfix.htm

In [9]:
# you may find the following precedence dictionary useful
prec = {'*': 2, '/': 2,
        '+': 1, '-': 1}

def infix_to_postfix(expr):
    """Returns the postfix form of the infix expression found in `expr`"""
    ops = Stack()
    postfix = []
    toks = expr.split()
    
    for i in toks:
        if i.isdigit():
            postfix.append(i)
        else:
            #if the stack is empty or contains a left parenthesis on top, push the token onto the stack.
            if ops.empty() or ops.peek()=='(':
                ops.push(i)
            
            #if the token is a left parenthesis, push it on the stack.
            elif i == '(':
                ops.push(i)
            
            #if the token is a right parenthesis, pop the stack and append all operators to the list until a left parenthesis is popped. Discard the pair of parentheses.
            elif i == ')':
                while True:
                    x = ops.pop()
                    if x == '(':
                        break
                    postfix.append(x)
            
            #if the token has higher precedence than the top of the stack, push it on the stack. For our purposes, the only operators are +, -, *, /, where the latter two have higher precedecence than the first two.
            elif prec[i] > prec[ops.peek()]:
                ops.push(i)

            #if the token has equal precedence with the top of the stack, pop and append the top of the stack to the list and then push the incoming operator.
            elif prec[i] == prec[ops.peek()]:
                x = ops.pop()
                postfix.append(x)
                ops.push(i)
            
            #if the incoming symbol has lower precedence than the symbol on the top of the stack, pop the stack and append it to the list. Then repeat the above tests against the new top of stack.
            elif prec[i] < prec[ops.peek()]:
                x = ops.pop()
                postfix.append(x)
                ops.push(i) #and push (not in instructions)
            
    #pop and append all operators on the stack to the list
    while ops.empty() is False:
        x = ops.pop()
        postfix.append(x)
    
    return ' '.join(postfix)

In [10]:
# (3 points)

from unittest import TestCase
tc = TestCase()
tc.assertEqual(infix_to_postfix('1'), '1')
tc.assertEqual(infix_to_postfix('1 + 2'), '1 2 +')
tc.assertEqual(infix_to_postfix('( 1 + 2 )'), '1 2 +')
tc.assertEqual(infix_to_postfix('1 + 2 - 3'), '1 2 + 3 -')
tc.assertEqual(infix_to_postfix('1 + ( 2 - 3 )'), '1 2 3 - +')

In [11]:
# (3 points)

from unittest import TestCase
tc = TestCase()
tc.assertEqual(infix_to_postfix('1 + 2 * 3'), '1 2 3 * +')
tc.assertEqual(infix_to_postfix('1 / 2 + 3 * 4'), '1 2 / 3 4 * +')
tc.assertEqual(infix_to_postfix('1 * 2 * 3 + 4'), '1 2 * 3 * 4 +')
tc.assertEqual(infix_to_postfix('1 + 2 * 3 * 4'), '1 2 3 * 4 * +')

In [12]:
# (3 points)

from unittest import TestCase
tc = TestCase()
tc.assertEqual(infix_to_postfix('1 * ( 2 + 3 ) * 4'), '1 2 3 + * 4 *')
tc.assertEqual(infix_to_postfix('1 * ( 2 + 3 * 4 ) + 5'), '1 2 3 4 * + * 5 +')
tc.assertEqual(infix_to_postfix('1 * ( ( 2 + 3 ) * 4 ) * ( 5 - 6 )'), '1 2 3 + 4 * * 5 6 - *')

## Queue

### 3. Circular, array-backed queue

In the following class, which you are to complete, the backing array will be created and populated with `None`s in the `__init__` method, and the `head` and `tail` indexes set to sentinel values (you shouldn't need to modify `__init__`). Enqueueing and Dequeueing items will take place at the tail and head, with `tail` and `head` tracking the position of the most recently enqueued item and that of the next item to dequeue, respectively. To simplify testing, your implementation should make sure that when dequeuing an item its slot in the array is reset to `None`, and when the queue is emptied its `head` and `tail` attributes should be set to `-1`.

Because of the fixed size backing array, the `enqueue` operation is defined to raise a `RuntimeError` when the queue is full — the same exception should be raised when `dequeue` is called on an empty queue.

Finally, the `resize` method will allow the array underlying the queue to be increased in size. It is up to you how to implement this (you can either leave the elements in their current positions, though this may require "unwrapping" elements, or you can simply move all elements towards the front of the array). You may assume that `resize` will only be called with a value greater than the current length of the underlying array.

In [1]:
class Queue:
    def __init__(self, limit=10):
        self.data = [None] * limit
        self.head = -1
        self.tail = -1

    def enqueue(self, val):
        #check self.head - self.tail is equal to 1
        if self.head - self.tail == 1:
            raise NotImplementedError
        #check len(self.data) - 1 is equal to self.tail
        if len(self.data) - 1 == self.tail and self.head == 0:
            raise NotImplementedError
        #check self.head is equal to -1
        if self.head == -1 and self.tail == -1:
            self.data[0] = val
            self.head = 0
            self.tail = 0
        else:
            #check len(self.data) - 1 is equal to self.tail
            if len(self.data) - 1 == self.tail and self.head != 0:
                self.tail = -1
            self.data[self.tail + 1] = val
            #increment the self.tail value
            self.tail += 1
        
    def dequeue(self):
        #check self.head is equal to self.tail
        if self.head == self.tail:
            temp = self.head
            self.head = -1
            self.tail = -1
            return self.data[temp]
        #check self.head is equal to -1
        if self.head == -1 and self.tail == -1:
            raise NotImplementedError
        #check self.head is not equal to len(self.data)
        if self.head != len(self.data):
            result = self.data[self.head]
            self.data[self.head] = None
            self.head = self.head + 1
        else:
           # resetting head value
            self.head = 0
            result = self.data[self.head]
            self.data[self.head] = None
            self.head = self.head + 1
        return result
    
    def resize(self, newsize):
        #check len(self.data) is less than newsize
        assert (len(self.data) < newsize)
        newdata = [None] * newsize
        head = self.head
        current = self.data[head]
        countValue = 0
        #Iterate the loop
        while current != None:
            newdata[countValue] = current
            countValue += 1
            #check countValue is not equal to 0
            if countValue != 0 and head == self.tail:
                break
            #check head is not equal to len(self.data) - 1
            if head != len(self.data) - 1:
                head = head + 1
                current = self.data[head]
            else:
                head = 0
                current = self.data[head]
        self.data = newdata
        self.head = 0
        self.tail = countValue - 1
    
    def empty(self):
        #check self.head is equal to -1
        # and self.tail is equal to -1
        if self.head == -1 and self.tail == -1:
            return True
        return False
    
    def __bool__(self):
        return not self.empty()
    
    def __str__(self):
        if not(self):
            return ''
        return ', '.join(str(x) for x in self)
    
    def __repr__(self):
        return str(self)
    
    def __iter__(self):
        head = self.head
        current = self.data[head]
        countValue = 0
        #Iterate the loop
        while current != None:
            yield current
            countValue += 1
            #check countValue is not equal to zero
            #check head is equal to self.tail
            if countValue != 0 and head == self.tail:
                break
            #check head is not equal to len(self.data) - 1
            if head != len(self.data) - 1:
                head = head + 1
                current = self.data[head]
            else:
                head = 0
                current = self.data[head]

In [2]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(5)
tc.assertEqual(q.data, [None] * 5)

for i in range(5):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(5)

for i in range(5):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())

In [3]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(10)

for i in range(6):
    q.enqueue(i)
    
tc.assertEqual(q.data.count(None), 4)

for i in range(5):
    q.dequeue()
    
tc.assertFalse(q.empty())
tc.assertEqual(q.data.count(None), 9)
tc.assertEqual(q.head, q.tail)
tc.assertEqual(q.head, 5)

for i in range(9):
    q.enqueue(i)

with tc.assertRaises(RuntimeError):
    q.enqueue(10)

for x, y in zip(q, [5] + list(range(9))):
    tc.assertEqual(x, y)
    
tc.assertEqual(q.dequeue(), 5)
for i in range(9):
    tc.assertEqual(q.dequeue(), i)

tc.assertTrue(q.empty())

In [4]:
# (5 points)

from unittest import TestCase
tc = TestCase()

q = Queue(5)
for i in range(5):
    q.enqueue(i)
for i in range(4):
    q.dequeue()
for i in range(5, 9):
    q.enqueue(i)
    
with tc.assertRaises(RuntimeError):
    q.enqueue(10)

q.resize(10)

for x, y in zip(q, range(4, 9)):
    tc.assertEqual(x, y)
    
for i in range(9, 14):
    q.enqueue(i)

for i in range(4, 14):
    tc.assertEqual(q.dequeue(), i)
    
tc.assertTrue(q.empty())
tc.assertEqual(q.head, -1)